In [85]:
%pip install pandas numpy scikit-learn joblib streamlit plotly

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\sukru\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [86]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib
from datetime import datetime, timedelta

In [ ]:
# Generate synthetic traffic data
def generate_traffic_data(n_samples=1000):
    np.random.seed(42)

    
    # Generate timestamps
    base_date = datetime(2025, 1, 1)
    timestamps = [base_date + timedelta(hours=i) for i in range(n_samples)]

    data = {
        'timestamp': timestamps,
        'day_of_week': [ts.weekday() for ts in timestamps],
        'hour': [ts.hour for ts in timestamps],
        'temperature': np.random.uniform(20, 45, n_samples),  # Warmer climate
        'precipitation': np.random.uniform(0, 50, n_samples),  # Monsoon rains
        'special_event': np.random.choice([0, 1], n_samples, p=[0.85, 0.15]),
        'road_work': np.random.choice([0, 1], n_samples, p=[0.80, 0.20]),
        'vehicle_count': np.random.poisson(1200, n_samples),  # Heavier traffic
    }

    df = pd.DataFrame(data)

    # Congestion modeling for Indian urban conditions
    base_congestion = (
        5 * np.sin(df['hour'] * np.pi / 12) +                         # Daily rhythm
        3 * (df['day_of_week'] < 6) +                                 # Mon–Sat traffic
        1.5 * ((df['hour'] >= 8) & (df['hour'] <= 10)) -              # Morning rush
        0.5*((df['hour'] > 10) & (df['hour'] <= 13)) +               # Midday lull
        5.5 * ((df['hour'] >= 17) & (df['hour'] <= 21)) +             # Evening rush
        0.07 * df['temperature'] +                                    # High heat adds discomfort
        0.05 * df['precipitation'] +                                  # Monsoon effect
        4 * df['special_event'] +                                     # Festivals/rallies
        3 * df['road_work'] +                                         # Road condition impact
        0.012 * (df['vehicle_count'] - 1200) +                        # Traffic volume influence
        np.random.normal(0, 0.6, n_samples)                         # Real-world noise
    )

    df['congestion_level'] = np.clip(base_congestion, 0, 10)
    return df

In [88]:
# Feature engineering
def prepare_features(df):
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)

    features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos',
                'temperature', 'precipitation', 'special_event',
                'road_work', 'vehicle_count']

    return df[features]

In [89]:
def main():
    print("Generating synthetic data...")
    df = generate_traffic_data()

    print("Preparing features...")
    X = prepare_features(df)
    y = df['congestion_level']

    print("Splitting and scaling data...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    print("Training model...")
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    print("Saving model and scaler...")
    joblib.dump(model, 'traffic_predictor_model.joblib')
    joblib.dump(scaler, 'traffic_scaler.joblib')

    print("Training complete! You can now run the Streamlit app.")

if __name__ == "__main__":
    main()

Generating synthetic data...
Preparing features...
Splitting and scaling data...
Training model...
Saving model and scaler...
Training complete! You can now run the Streamlit app.
